In this lab, we are going to create a tool and use it in our agentic AI solution. The tool will provide two functions.
1. Record a message a user on your website might want to leave for you (along with user email etc)
2. Record a question the agentic AI is not able to answer

For both these functions, we will capture the information and push it directly to your phone using a pushover app.
you need to install it on your phone and also visit pushover.net to generate API key for USER and for the APP.
As usual this information will be stored in the .env file.

In [8]:
import os
from dotenv import load_dotenv
import json
from openai import OpenAI
import gradio as gr
from pypdf import PdfReader
import requests


In [ ]:
load_dotenv(override=True)
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
if any(x is None for x in [pushover_user, pushover_token]):
    print("PUSHOVER_USER or PUSHOVER_TOKEN not found in .env file")

In [ ]:
# just test a quick pushover message.
def push(message):
    payload = {'user': pushover_user, 'token': pushover_token, 'message': message}
    r = requests.post('https://api.pushover.net/1/messages.json', data=payload)
    print(r.json())

push('Hello from Pushover!')

In [10]:
# these are the two functions we will use in our tool.
def record_user_details(email, name='Not provided', notes='Not provided'):
    push(f"recording interest from '{name}' with email '{email}'. Notes: '{notes}'")
    return {'recorded': 'ok'}

def record_unknown_question(question):
    push(f"recording question: '{question}' that I could not answer")
    return {'recorded': 'ok'}

In [11]:
# This is some boilerplate json to define the name of the tool, the description and the parameters it requires.
record_user_details_json = {
    "name": "record_user_details", 
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [12]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [13]:
# This is the list of tools that the agent can use.
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [14]:
# This function can take a list of tool calls, and run them. This is the IF statement!!
# Remember, we had said tools are nothing more than some json and some if conditions.

# tool_calls is an object
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        # this is the result which will be returned to the agent. Note the "role" is "tool", not "user" or "system"
        # also we return a list of results.
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
# pythonic way of converting the string to a function with the same name.
# globals is a dictionary of all the global variables in the current scope.
globals()["record_unknown_question"]("this is a really hard question") # this will push a message to your phone.

In [16]:
# tool_calls is an object
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
# now prepare to read the linkedin.pdf and the summary.txt file as in the previous lab.
name = 'Rajat Girotra'
summary = ''
with open('summary.txt', 'r', encoding="utf-8") as f:
    summary = f.read()

linkedin = ''
reader = PdfReader('linkedin.pdf')
for page in reader.pages:
    text = page.extract_text()
    linkedin += text

print(summary)
print(linkedin)


In [18]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [19]:
openai_python_client = OpenAI()

In [22]:
# this is the most important function
# here we will parse the user message and see if we need a tool call and how to make it and get results back.
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        # send message to agent
        response = openai_python_client.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        print(f"finish_reason: {finish_reason}")

        # finish_reason is used to check if LLM wants a tool call.
        if finish_reason == 'tool_calls':
            # we dont get the content from the message object. infact we get the tool_calls object
            msg = response.choices[0].message
            tool_calls = msg.tool_calls
            print(f"type(tool_calls): {type(tool_calls)}")
            results = handle_tool_calls(tool_calls)
            # why we do this? We are creating a new prompt for the LLM with the result of the tools call.
            # if the tool call is successful, the LLM will see that and in the second iteration, the finish_reason will not be 'tool_calls'
            # it will just be 'stop'
            messages.append(msg) 
            messages.extend(results) 
            print(f"extended messages: {messages}")
        else:
            done = True
    return response.choices[0].message.content


# A sample message i.e. ChatCompletionMessage which has many attributes using tool_calls which is a list of ChatCompletionMessageToolCall, ie list of tool calls we need to make.
# that is why we loop through the tool_calls and call the handle_tool_calls function to make all the tool calls. and in the result we append the tool call id.
# msg = ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8G9yjI0aiKljPzFXJGvStOAu', function=Function(arguments='{"question":"who are you?"}', name='record_unknown_question'), type='function')]), 

In [ ]:
gr.ChatInterface(chat, type="messages").launch()